In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-01-31 13:56:02--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8002::154, 2606:50c0:8000::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.09s   

2024-01-31 13:56:03 (12.1 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [2]:
with open('./input.txt', 'r') as f:
    text = f.read()

In [3]:
len(text)

1115394

In [4]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f"Vocab size: {vocab_size}, for {''.join(chars)}")


Vocab size: 65, for 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [6]:
#Tokenization strategy
# convert vocab to int and vice versa
stoi= {ch:i for i,ch in enumerate(chars)}
itos= {i:ch for i,ch in enumerate(chars)}
#encoder: take string of chars, output a list of integers
encode = lambda s: [stoi[c] for c in s]
#decoder: take list of integer, output the string
decode = lambda l: ''.join([itos[i] for i in l])


print(encode('shirin'))
print(decode([57, 46, 47, 56, 47, 52]))



[57, 46, 47, 56, 47, 52]
shirin


In [65]:
import tiktoken #different encoder, used in OpenAI, BPE
enc = tiktoken.get_encoding('gpt2')
enc.n_vocab 
enc.encode('Hi Shiirin!')
enc.decode([686,4555])

' ro eth'

### get the encoding of our entire dataset and save it in torch.tensor format

In [28]:
import torch
data =torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


### split the dataset into train and test sets (90-10)

In [29]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]
#len(val_data v )

### block size for batch processing

In [34]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [37]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'when input is {context} what we are tryin to predict is {target}')

when input is tensor([18]) what we are tryin to predict is 47
when input is tensor([18, 47]) what we are tryin to predict is 56
when input is tensor([18, 47, 56]) what we are tryin to predict is 57
when input is tensor([18, 47, 56, 57]) what we are tryin to predict is 58
when input is tensor([18, 47, 56, 57, 58]) what we are tryin to predict is 1
when input is tensor([18, 47, 56, 57, 58,  1]) what we are tryin to predict is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) what we are tryin to predict is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) what we are tryin to predict is 58


### Batch size

In [53]:
torch.manual_seed(1337)
batch_size = 4 #how many independent sequences will we process in each forward/backward passes
block_size = 8 #max context len for prediction

def get_batch(split): #generate smal batch of data of input x n target y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)- block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size]for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    return x, y

In [57]:
xb,yb = get_batch('train')
print('inputs:')
print(xb, xb.shape)
print('targets:')
print(yb, yb.shape)

print('-----')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b,t]

    print(f'when input context is {context.tolist()} the target is {target}')


inputs:
tensor([[50, 39, 52, 63,  1, 47, 58, 57],
        [56, 53, 63,  1, 42, 47, 42,  1],
        [39, 51,  1, 39, 44, 56, 39, 47],
        [17, 24, 21, 38, 13, 14, 17, 32]]) torch.Size([4, 8])
targets:
tensor([[39, 52, 63,  1, 47, 58, 57, 43],
        [53, 63,  1, 42, 47, 42,  1, 57],
        [51,  1, 39, 44, 56, 39, 47, 42],
        [24, 21, 38, 13, 14, 17, 32, 20]]) torch.Size([4, 8])
-----
when input context is [50, 39, 52, 63, 1, 47, 58, 57] the target is 43
when input context is [56, 53, 63, 1, 42, 47, 42, 1] the target is 57
when input context is [39, 51, 1, 39, 44, 56, 39, 47] the target is 42
when input context is [17, 24, 21, 38, 13, 14, 17, 32] the target is 20


In [58]:
print(xb)

tensor([[50, 39, 52, 63,  1, 47, 58, 57],
        [56, 53, 63,  1, 42, 47, 42,  1],
        [39, 51,  1, 39, 44, 56, 39, 47],
        [17, 24, 21, 38, 13, 14, 17, 32]])


## Simple Bigram Model


In [ ]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

